# Indexed Priority Queue (D-Ary Heap)

**Indexed priority queues** have a wide range of real world applications: from computer science and mathematics to machine learning and biology and logistics and even more.

They are very efficient Abstract Data Structures which allow to perform operations such as look-up of elements, polling, indexing, updating and others to be run in **logarithmic  O(log(n)** and **constant O(n)** time.

In [1]:
class IPQ:
    def __init__(self):
        self.debug_swap = False
        self._size = 0
        self._num_elem = 0
        self._degree = 0
        self._child = []
        self._parent = []
        self.ki_count = 0
        # The Position map (pos_map) to map Key Indexes (ki) to where the position of that
        # key is represented in the priority queue in the domain (0, sz)
        self.pos_map = []
        
        # The Inverse Map (inv_map) stores the indexes of the keys in the range (0, sz)
        # which make up the priority queue. It should be noted that 'im' and 'pm'
        # are inverses of each other, so: pm[im[i]] = im[pm[i]] = i
        self.inv_map = []
        
        # The values associated with the keys. It is very importantt to note
        # that this array is indexed by the key indexes(aka 'ki')
        self.values = []
   
    def min_heap(self, degree, max_size):
        self._degree = max(2, degree)
        self._num_elem = max(self._degree + 1, max_size)
        
        self.ki_arr = [j for j in range(self._size)] 
        self.pos_map = [0] * self._num_elem
        self.inv_map = [0] * self._num_elem
        self._child = [0] * self._num_elem
        self._parent = [0] * self._num_elem
        self.values = [None] * self._num_elem
        
        for i in range(self._num_elem):
            self._parent[i] = (i - 1) // self._degree    
            self._child[i] = i * self._degree + 1
            self.pos_map[i] = self.inv_map[i] = -1

    def size(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def contains(self, ki):
        self.key_inbounds_or_raise(ki)
        return self.pos_map[ki] != -1
      
    def peek_min_key_index(self):
        self.is_not_empty_or_raise()
        return self.inv_map[0]
    
    def poll_min_key_index(self):
        minkey = self.peek_min_key_index() 
        self.delete(minkey)
        return minkey
     
    def peek_min_value(self):
        self.is_not_empty_or_raise()
        return self.values[self.inv_map[0]]
          
    def poll_min_value(self):
        min_value = self.peek_min_value()
        self.delete(self.peek_min_key_index())
        return min_value
    
    def insert(self, value, ki=None):
        if ki is None:
            ki = self.ki_count
            self.ki_arr.append(self.ki_count)
            self.ki_count += 1
        '''by default inserts at ki=_size'''
        if ki + 1 > self._num_elem:
            raise IndexError(f'Key index out of bounds; recieved: {ki}')
        if self.contains(ki):
            raise ValueError(f'Index already exists; recieved: {ki}')
        self.value_not_None_or_raise(value)
        self.pos_map[ki] = self._size
        self.inv_map[self._size] = ki
        self.values[ki] = value
        self.swim(self._size)
        self._size += 1  #v1
          
    def value_of(self, ki):
        self.key_exists_or_raise(ki)
        return self.values[ki]
    
    def delete(self, ki):
        self.key_exists_or_raise(ki)
        i = self.pos_map[ki]
        self._size -= 1 
        self.swap(i, self._size)
        self.sink(i)
        self.swim(i)
        value = self.values[ki] 
        self.values[ki] = None
        self.pos_map[ki] = -1
        self.inv_map[self._size] = -1
        return value 
    
    def sink(self, i):
        j = self.min_child(i)
        while j != -1 and self.values[self.inv_map[i]] > self.values[self.inv_map[j]]:
            tmp1 = j
            self.swap(i, j)
            i = tmp1
            j = self.min_child(i)
            
    def update(self, ki, value):
        self.key_exists_and_value_not_None_or_raise(ki, value)
        i = self.pos_map[ki]
        old_value = self.values[ki]
        self.values[ki] = value
        self.sink(i)
        self.swim(i)
        return old_value

    def decrease(self, ki, value):
        self.key_exists_and_value_not_None_or_raise(ki, value)
        if self.less(value, self.values[ki], is_value=True):
            self.values[ki] = value
            self.swim(self.pos_map[ki])

    def increase(self, ki, value):
        self.key_exists_and_value_not_None_or_raise(ki, value)
        if self.less(self.values[ki], value, is_value=True):
            self.values[ki] = value
            self.sink(self.pos_map[ki])
                     
    ''' Helper functions '''
    def swim(self, i):

        while self.less(i, self._parent[i]) and i > 0:
            temp = self._parent[i]
            self.swap(i, self._parent[i])
            i = temp

    def swap(self, heap_i, heap_j):
        '''
        Swaps node i with node j in pos_map and inv_map to maintain
        heap invariant. Used in delete, sink and swim functions.
        
        Args:
        - heap_i: Heap's node index
        - heap_j: Heap's node index
        
        Returns: None
        '''        
        if self.debug_swap:
            print('\n======= DEBUG: swap =======')
            print('---------------------------')
            print('\tBefore swap:')
            print('\tkey_arr:', self.ki_arr)
            print('\tval_arr:', self.values)
            print('\tpos_map:', self.pos_map)
            print('\tinv_map:', self.inv_map)
            
        ki_i = self.inv_map[heap_i]
        ki_j = self.inv_map[heap_j]
        self.inv_map[heap_i], self.inv_map[heap_j] = self.inv_map[heap_j], self.inv_map[heap_i]
        self.pos_map[ki_i], self.pos_map[ki_j] = self.pos_map[ki_j], self.pos_map[ki_i]

        if self.debug_swap:
            print('\n\tAfter swap:')
            print('\tkey_arr:', self.ki_arr)
            print('\tval_arr:', self.values)
            print('\tpos_map:', self.pos_map)
            print('\tinv_map:', self.inv_map)
            print('---------------------------\n')

    
    # From the parent node at index [i] find the minimum child below it                 
    def min_child(self, i):
        self._children = {}
        index = -1 
        frm = self._child[i] 
        to = min(self._size, (frm + self._degree))
        for j in range(frm, to):    
            self._children[j] = self.values[self.inv_map[j]]
        for k, v in self._children.items():
            if v == min(self._children.values()):
                index = k
        return index 
    
    ''' Tests if the value of node [i] < node [j] '''
    def less(self, i, j, is_value=False):
        
        # If values are passed directly for comparison
        if is_value:
            if i < j: return True
            else: return False
        # If  heap nodes are passed for comparison
        else: 
            if self.values[self.inv_map[j]] is None: return False
        if self.values[self.inv_map[i]] < self.values[self.inv_map[j]]:
            return True
        else: return False
        
    ''' Helper functions to make the code more readable '''
    def is_not_empty_or_raise(self):
        if self.is_empty(): raise ValueError("Priority queue underflow")

    def key_exists_and_value_not_None_or_raise(self, ki, value):
        self.key_exists_or_raise(ki)
        self.value_not_None_or_raise(value)

    def key_exists_or_raise(self, ki):
        if not self.contains(ki): raise ValueError(f'Index does not exist; received: {ki}')
     
    def value_not_None_or_raise(self, value):
        if value == None: raise ValueError('Value cannot be None')

    def key_inbounds_or_raise(self, ki):
        if ki < 0 or ki > self._num_elem:
            raise ValueError(f'Key index out of bounds; recieved: {ki}')
   
    ''' Test functions '''
    # Recursively checks if this heap is a min heap. This method is used
    # for testing purposes to validate the heap invariant
    def is_min_heap(self):
        return self._is_min_heap(0)
    
    def _is_min_heap(self, i):
        frm = self._child[i]
        to = min(self._size, frm + self._degree)
        for j in (frm, to):
            if j > self._size: continue
            if not self.less(i, j): return False
            if not self._is_min_heap(j): return False
        
        return True

In [2]:
# Creating Class object

ipq = IPQ()
ipq.min_heap(2, 10)

ipq.insert(5)
ipq.insert(3)
ipq.insert(6)
ipq.insert(1)
ipq.insert(4)
ipq.insert(7)
ipq.insert(0)
ipq.insert(8)

In [3]:
print(f'KEYS :   {ipq.ki_arr} \n          \u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3---\u21D3\nvalues:  { ipq.values}\npos_map: {ipq.pos_map}\ninv_map: {ipq.inv_map}')

KEYS :   [0, 1, 2, 3, 4, 5, 6, 7] 
          ⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓---⇓
values:  [5, 3, 6, 1, 4, 7, 0, 8, None, None]
pos_map: [3, 1, 6, 2, 4, 5, 0, 7, -1, -1]
inv_map: [6, 1, 3, 0, 4, 5, 2, 7, -1, -1]


In [4]:
# Inserting Values

ipq.insert(11)
print(f'KEYS :   {ipq.ki_arr} \n          \u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3---\u21D3\nvalues:  { ipq.values}\npos_map: {ipq.pos_map}\ninv_map: {ipq.inv_map}')

KEYS :   [0, 1, 2, 3, 4, 5, 6, 7, 8] 
          ⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓---⇓
values:  [5, 3, 6, 1, 4, 7, 0, 8, 11, None]
pos_map: [3, 1, 6, 2, 4, 5, 0, 7, 8, -1]
inv_map: [6, 1, 3, 0, 4, 5, 2, 7, 8, -1]


In [5]:
# Deleting Values

ipq.delete(8)
print(f'KEYS :   {ipq.ki_arr} \n          \u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3---\u21D3\nvalues:  { ipq.values}\npos_map: {ipq.pos_map}\ninv_map: {ipq.inv_map}')

KEYS :   [0, 1, 2, 3, 4, 5, 6, 7, 8] 
          ⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓---⇓
values:  [5, 3, 6, 1, 4, 7, 0, 8, None, None]
pos_map: [3, 1, 6, 2, 4, 5, 0, 7, -1, -1]
inv_map: [6, 1, 3, 0, 4, 5, 2, 7, -1, -1]


In [6]:
# Updating Values

ipq.update(6,42)
print(f'KEYS :   {ipq.ki_arr} \n          \u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3--\u21D3---\u21D3\nvalues:  { ipq.values}\npos_map: {ipq.pos_map}\ninv_map: {ipq.inv_map}')

KEYS :   [0, 1, 2, 3, 4, 5, 6, 7, 8] 
          ⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓--⇓---⇓
values:  [5, 3, 6, 1, 4, 7, 42, 8, None, None]
pos_map: [3, 1, 2, 0, 4, 5, 6, 7, -1, -1]
inv_map: [3, 1, 2, 0, 4, 5, 6, 7, -1, -1]
